In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.optimizers import SGD, Adam
from keras.layers import *
from keras.models import Sequential, Model
from sklearn.model_selection import *
from sklearn.metrics import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import MobileNet

In [3]:
def load_images():
  path = "Dataset\PCOSGen-train\PCOSGen-train\images"
  images = os.listdir(path)
  x = []
  for img in images:
    temp = cv2.imread(os.path.join(path ,img))
    twmp=cv2.resize(temp,(256,256))

    x.append(temp)

  return x

In [4]:
df = pd.read_excel("Dataset\class_label.xlsx")
df.head()

,imagePath,Healthy
0,100image13.jpg,1
1,100image2.jpg,1
2,100image65.jpg,0
3,100image71.jpg,0
4,100image83.jpg,0


In [4]:
##ANN model

def ann():
  model=Sequential()
  model.add(Flatten(input_shape=(256,256,3)))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  return model


In [16]:
def mobilenet(input_shape=(256,256,3)):
  base_model=MobileNet(weights='imagenet',include_top='False', input_shape=input_shape)
  model=Sequantial()
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [5]:
image_names = df['imagePath'].values
df_filtered = df[df['imagePath'].isin(image_names)]


In [6]:
x=load_images()

In [7]:
images = []
labels = []
for index, row in df_filtered.iterrows():
    image_name = row['imagePath']
    label = row['Healthy']
    image_index = np.where(np.array(image_names) == image_name)[0][0]
    image = x[image_index]
    resized_image = cv2.resize(image, (256, 256))
    images.append(resized_image)
    labels.append(label)

In [8]:
images = np.array(images)
labels = np.array(labels)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [10]:
y_train_reshaped = np.expand_dims(np.expand_dims(y_train, axis=-1), axis=-1)
y_test_reshaped = np.expand_dims(np.expand_dims(y_test, axis=-1), axis=-1)

batch_size=16
epochs=10


In [20]:
# print("Training ANN model...")

# # Define batch size and number of epochs
# batch_size = 32
# epochs = 10  # Adjust as needed

# ann_model=ann()
# ann_model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

# # Iterate over the dataset in batches
# for epoch in range(epochs):
#     print(f"Epoch {epoch+1}/{epochs}")
#     for batch_index in range(0, len(x_train), batch_size):
#         x_batch = x_train[batch_index:batch_index+batch_size]
#         y_batch = y_train[batch_index:batch_index+batch_size]
        
#         # Train on the current batch
#         ann_model.train_on_batch(x_batch, y_batch)
    
#     # Evaluate the model at the end of each epoch
#     loss, accuracy = ann_model.evaluate(x_test, y_test)
#     print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# print("Training finished.")

Training ANN model...


Epoch 1/10



20/20 [==============================] - 9s 69ms/step - loss: 0.5924 - accuracy: 0.7391
Validation Loss: 0.5923860669136047, Validation Accuracy: 0.739062488079071
Epoch 2/10
20/20 [==============================] - 3s 113ms/step - loss: 0.5768 - accuracy: 0.7391
Validation Loss: 0.5767970085144043, Validation Accuracy: 0.739062488079071
Epoch 3/10
20/20 [==============================] - 2s 89ms/step - loss: 0.5756 - accuracy: 0.7391
Validation Loss: 0.5756357312202454, Validation Accuracy: 0.739062488079071
Epoch 4/10


KeyboardInterrupt: 

In [46]:
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

class_labels=np.unique(y_train)
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=class_labels, y=y_train)
class_weights=dict(zip(np.unique(y_train), class_weights)),class_weights

In [40]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1.0 / 255  # Rescale pixel values to [0, 1]
)

In [37]:
def custommodel():
    model = Sequential()
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [38]:
model=custommodel()
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [29]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [14]:
steps_per_epoch = len(x_train) // batch_size
batch_size = 16  # Reduce batch size to conserve memory
epochs = 10

In [49]:
history = model.fit_generator(
    generator=datagen.flow(x_train[:500], y_train[:500], batch_size=batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping],
)

C:\Users\hp\AppData\Local\Temp\ipykernel_15060\2761170086.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
32/80 [===========>..................] - ETA: 27s - loss: 0.5933 - accuracy: 0.7320WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 800 batches). You may need to use the repeat() function when building your dataset.


80/80 [==============================] - 32s 372ms/step - loss: 0.5933 - accuracy: 0.7320 - val_loss: 52.8794 - val_accuracy: 0.7391


In [1]:
from keras.applications.densenet import DenseNet121


In [11]:
# Convert labels to categorical
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, BatchNormalization, Activation, Concatenate, AveragePooling2D
from keras.applications.densenet import DenseNet121

y_traincatg = to_categorical(y_train, num_classes=2)
y_testcatg = to_categorical(y_test, num_classes=2)

In [24]:
def dense():
    base_model = DenseNet121(include_top=False, weights=None, input_shape=(256, 256, 3))

    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [25]:
batch_size = 16
epochs = 10
model=dense()
Momentum = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=Momentum, loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
# for epoch in range(epochs):
#     print(f"Epoch {epoch+1}/{epochs}")
#     for batch_index in range(0, len(x_train), batch_size):
#         x_batch = x_train[batch_index:batch_index+batch_size]
#         y_batch = y_train[batch_index:batch_index+batch_size]
        
#         # Train on the current batch
#         model.train_on_batch(x_batch, y_batch)
    
#     # Evaluate the model at the end of each epoch
#     loss, accuracy = model.evaluate(x_test, y_test)
#     print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# print("Training finished.")

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=16)


Epoch 1/10
 14/160 [=>............................] - ETA: 1:40:34 - loss: 9.7803 - accuracy: 0.6116

KeyboardInterrupt: 

In [29]:
def densecateg():
    base_model = DenseNet121(include_top=False, weights=None, input_shape=(256, 256, 3))

    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [30]:
batch_size = 16
epochs = 10
model=densecateg()
Momentum = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=Momentum, loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
# for epoch in range(epochs):
#     print(f"Epoch {epoch+1}/{epochs}")
#     for batch_index in range(0, len(x_train), batch_size):
#         x_batch = x_train[batch_index:batch_index+batch_size]
#         y_batch = y_traincatg[batch_index:batch_index+batch_size]
        
#         # Train on the current batch
#         model.train_on_batch(x_batch, y_batch)
    
#     # Evaluate the model at the end of each epoch
#     loss, accuracy = model.evaluate(x_test, y_testcatg)
#     print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# print("Training finished.")

model.fit(x_train[:500], y_traincatg[:500], validation_data=(x_test, y_testcatg), epochs=10, batch_size=16)


Epoch 1/10
